# Group $\mathsf{G}_4$

Given a combinatorial type $\mathsf{w}$, let $\Sigma_{\mathsf{Br}}$ be the subcomplex of $\mathsf{TS}(\mathsf{w})$ obtained by removing each branch. The combinatorial types $\mathsf{w}$ belonging to $\mathsf{G}_4$ are those not in $\mathsf{G}_i$ ($i\leq 3$), and such that $\Sigma_{\mathsf{Br}}$ is vertex-connecting. 

In [ ]:
using Oscar
using Combinatorics
pm = Polymake

In [ ]:
cd("..")

In [ ]:
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/tscCoordRing.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));
include(joinpath(currentDir, "src/Bmaximal.jl"));
include(joinpath(currentDir, "src/simplifyIdeal.jl"));

The list ```G4``` contains all combinatorial types in $\mathsf{G}_4$. 

In [ ]:
G4Path = joinpath(currentDir,"groupsFinal/G4.dat")
G4 = file2SetVectors(G4Path);

---
**Function**: ```branchDecomposition(graphOscar::Graph)```

*Description*: 

*Example*: 

---
**Function**: ```removeBranchCVP(w::Vector{Int64})```

*Description*: 

*Example*: 



In [ ]:
function branchDecomposition(dg)
    g = n_edges(dg) - n_vertices(dg) + 1
    if g==0
        return "tree"
    end
    
    BrVerts = leavesGraph(dg)
    BrEdges = [(i, first(all_neighbors(dg, i))) for i in BrVerts]
    candidates = unique!([e[2] for e in BrEdges])
        
    stop = false
    i=0
    
    while !stop
        D = Dict([i => [j for j in all_neighbors(dg,i) if !(j in BrVerts)] for i in candidates])     
        newCandidates = [i for i in keys(D) if length(D[i]) == 1]
        if length(newCandidates) == 0
            stop = true
        else
            unique!(append!(BrVerts, newCandidates))
            unique!(append!(BrEdges, [(i, first(D[i])) for i in newCandidates]))
            precandidates = unique!([e[2] for e in BrEdges])   
            candidates = [i for i in precandidates if !(i in BrVerts)]
        end
        i+=1
    end
    Center = [i for i in 1:n_vertices(dg) if !(i in BrVerts) ]
    return (BrVerts, BrEdges, Center)
end

function removeBranchCVP(w)
    subd = subdivision_of_points(vDelta38[:,2:9], -w)
    GrO = subd2DualGraph(subd)
    Ms = subd2Matroids(subd,3,8)
    g = n_edges(GrO) - n_vertices(GrO) + 1
    if g == 0
        return "dual graph is tree"
    end
    BrV, BrE, C = branchDecomposition(GrO)
    commonB = commonBasis([Ms[i] for i in C])
    return length(commonB) > 0    
end

We verify that all branches are paths. 

In [ ]:
branches_are_paths = []
branches_are_not_all_paths = []

for w in G4
    subd = subdivision_of_points(vDelta38[:,2:9], -w)
    GrO = subdDualGraph(subd)
    BrV, BrE, C = branchDecomposition(GrO)
    countBrNeighbors = [all_neighbors(GrO,v) for v in BrV]
    if all([length(n)<3 for n in countBrNeighbors])
        push!(branches_are_paths, w)
    else
        push!(branches_are_not_all_paths, w)
    end
end

In [ ]:
length(branches_are_not_all_paths) == 0

For a given combinatorial type $\mathsf{w}$, we determine $\Sigma_L$, and compute its coordinate ring using Proposition 6.3, implemented by the function ```limitTSC```. Then we determine those whose ideal $I_{\Sigma_L}$ satisfies the hypotheses of Lemma 6.11. 

In [ ]:
hasUpperTriangularG4 = []
doesntHaveUpperTriangularG4 = []

R, x = makePolyRing(3, 8, QQ)

UTFile = joinpath(currentDir,"G4Data/hasUpperTriangularG4.dat")
NUTFile = joinpath(currentDir,"G4Data/doesnthaveUpperTriangularG4.dat")

for w in G4
    subd = subdivision_of_points(vDelta38[:,2:9], -w)
    GrO = subdDualGraph(subd)
    
    (BrVerts, BrEdges, Center) = branchDecomposition(GrO)
    
    Ms = subd2Matroids(subd,3,8)
    MsCenter = [Ms[i] for i in Center]
    
    optBs = optimalBasesForLimit(MsCenter)
    Lim = limitTSC(MsCenter, QQ, first(optBs), R, x)
    
    Rt = base_ring(Lim)
    xt = gens(base_ring(Lim))
    gLim = minimal_generating_set(Lim)
    
    if systemHasUniUpperTriangle(gLim, Rt, xt)
        push!(hasUpperTriangularG4, w)
        open(UTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;
    else
        push!(doesntHaveUpperTriangularG4, w)
        open(NUTFile, "a") do io
            write(io, vec2String(w), "\n")
        end;
    end
end

We appply Algorithm 6.12 to the combinatorial types in ```doesntHaveUpperTriangularG4```. If the output ideal is $\langle 0 \rangle$, then the function ```canReduceIdeal``` returns ```true```. In this case, the ring $R_{\Sigma_{L}}$ is a regular integral domain. 

In [ ]:
canFullySimplifyG4 = []
cannotFullySimplifyG4 = []

canFile = joinpath(currentDir, "G4Data/canFullySimplifyG4.dat")
cannotFile = joinpath(currentDir, "G4Data/cannotFullySimplifyG4.dat")

io1 = open(canFile, "w") 
io2 = open(cannotFile, "w")
close(io1)
close(io2)

R, x = makePolyRing(3,8, QQ)

for w in doesntHaveUpperTriangularG4
    subd = subdivision_of_points(vDelta38[:,2:9], -w)
    GrO = subdDualGraph(subd)
    
    (BrVerts, BrEdges, Center) = branchDecomposition(GrO)
   
    Ms = subd2Matroids(subd,3,8)
    MsCenter = [Ms[i] for i in Center]
    
    optBs = optimalBasesForLimit(MsCenter)
    Lim = limitTSC(MsCenter, QQ, first(optBs), R, x)
        
    Rt = base_ring(Lim)
    xt = gens(base_ring(Lim))
    gLim = minimal_generating_set(Lim)
    
    Sinv = localizingSemiGroup(MsCenter, QQ, first(optBs), R, x)
    SR = localization(Sinv)[1]
    
    if canReduceIdeal(gLim, R, x, Sinv, SR)
        push!(canFullySimplifyG4, w)
        open(canFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    else
        push!(cannotFullySimplifyG4, w)
        open(cannotFile, "a") do io
           write(io, vec2String(w), "\n")
        end;
    end
end

As we see, ```canReduceIdeal``` returns true for all of the combinatorial types in ```doesntHaveUpperTriangularG4```.

In [ ]:
#cannotFullySimplifyG4 = file2SetVectors(joinpath(currentDir, "G4Data/cannotFullySimplifyG4.dat"));
print(length(cannotFullySimplifyG4) == 0)

## Dimension of $\operatorname{Gr}(\mathsf{w})$

Next we check that the dimensions $\operatorname{Gr}(\mathsf{w})$ are 15 for each $\mathsf{w}$ in $\mathsf{G}_4$. We do this by applying Formula ??, which is implemented in the function ```dimensionBranchCVP``` below. 


In [ ]:
function matroidOfEdge(Ms,e,n)
    B1 = bases(Ms[e[1]])
    B2 = bases(Ms[e[2]])
    return matroid_from_bases(intersect(B1,B2), n)
end

function dimensionBranchCVP(w, F, d, n, R, x)
    subd = subdivision_of_points(vDelta38[:,2:9], -w)
    GrO = subdDualGraph(subd)
    Ms = subd2Matroids(subd,d,n)
    
    g = n_edges(GrO) - n_vertices(GrO) + 1
    
    if g == 0
        return "dual graph is tree"
    end
    
    (BrVerts, BrEdges, Center) = branchDecomposition(GrO)
        
    dimsBrV = [dimTSC_Optimized(Ms[i], F, R, x) for i in BrVerts] 
    dimsBrE = [dimTSC_Optimized(matroidOfEdge(Ms,e,n), F, R, x) for e in BrEdges]
    
    dimCenter = dimLimitTSC([Ms[i] for i in Center], F, R, x)
    
    return sum(dimsBrV) - sum(dimsBrE) + dimCenter
    
end;

In [ ]:
R, x =  makePolyRing(3,8,QQ)
dims_G4 = []

dimFile = joinpath(currentDir, "G4Data/dims_G4_test.dat")
io1 = open(dimFile, "w") 
close(io1)

for i in 1:length(G4)
    w = G4[i]
    d = dimensionBranchCVP(w, QQ, 3, 8, R, x)
    push!(dims_G4, d)
    
    open(dimFile, "a") do io
        write(io, string("dim w", i, " = " , d, "\n"))
        end
end

In [ ]:
all([d == 15 for d in dims_G4])